In [58]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text  import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense,Bidirectional,LSTM
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text  import Tokenizer,one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
messages = pd.read_csv("imdb_dataset.csv")

# messages = messages.iloc[:20,:]

corpus = []
WL =  WordNetLemmatizer()
for index in range(len(messages)):
  if index%1000==0:
     print(f"running index is {index}")
  html_tag_remover = re.sub("<.*>"," ",messages["review"][index])
  filtered_words = re.sub("\W"," ",html_tag_remover)
  message = filtered_words.lower()
  words_lst = [WL.lemmatize(word) for word in nltk.word_tokenize(message) if word not in stopwords.words('english')]
#   words_lst = [PS.stem(word) for word in nltk.word_tokenize(message) if word not in stopwords.words('english')]
  document = " ".join(words_lst)
  corpus.append(document)

running index is 0
running index is 1000
running index is 2000
running index is 3000
running index is 4000
running index is 5000
running index is 6000
running index is 7000
running index is 8000
running index is 9000
running index is 10000
running index is 11000
running index is 12000
running index is 13000
running index is 14000
running index is 15000
running index is 16000
running index is 17000
running index is 18000
running index is 19000
running index is 20000
running index is 21000
running index is 22000
running index is 23000
running index is 24000
running index is 25000
running index is 26000
running index is 27000
running index is 28000
running index is 29000
running index is 30000
running index is 31000
running index is 32000
running index is 33000
running index is 34000
running index is 35000
running index is 36000
running index is 37000
running index is 38000
running index is 39000
running index is 40000
running index is 41000
running index is 42000
running index is 43000
r

In [3]:
le = LabelEncoder()
order =  ["negative","positive"]
le.fit(order)
Y=le.transform(messages['sentiment'])

X_train,X_test,Y_train,Y_test = train_test_split(corpus,Y,test_size=0.2,random_state=4)
# ##Using onehot vector representation for converting the sequences into vectors.

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
vocab_size = len(word_index)+1
sent_length = 100
emb_dim = 100
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences,padding="post",maxlen=sent_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences,padding="post",maxlen=sent_length)

In [4]:
model = Sequential()

model.add(Embedding(input_dim=vocab_size,output_dim=emb_dim,input_length=sent_length))
model.add(Bidirectional(LSTM(units=128)))
model.add(Dense(64,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model.fit(train_padded,Y_train,epochs=10,verbose=1)
prediction = model.predict(test_padded)

print(prediction[:10])
for value in prediction:
    if value[0]>0.5:
        value[0]=1
    else:
        value[0]=0
print(accuracy_score(Y_test,prediction))
print(classification_report(Y_test,prediction))

Epoch 1/10
1250/1250 [==============================] - 119s 94ms/step - loss: 0.3927 - accuracy: 0.8204
Epoch 2/10
1250/1250 [==============================] - 113s 91ms/step - loss: 0.2052 - accuracy: 0.9201
Epoch 3/10
1250/1250 [==============================] - 114s 91ms/step - loss: 0.1184 - accuracy: 0.9559
Epoch 4/10
1250/1250 [==============================] - 119s 95ms/step - loss: 0.0683 - accuracy: 0.9752
Epoch 5/10
1250/1250 [==============================] - 115s 92ms/step - loss: 0.0397 - accuracy: 0.9865
Epoch 6/10
1250/1250 [==============================] - 118s 94ms/step - loss: 0.0346 - accuracy: 0.9883
Epoch 7/10
1250/1250 [==============================] - 116s 93ms/step - loss: 0.0216 - accuracy: 0.9926
Epoch 8/10
1250/1250 [==============================] - 116s 93ms/step - loss: 0.0164 - accuracy: 0.9948
Epoch 9/10
1250/1250 [==============================] - 117s 94ms/step - loss: 0.0165 - accuracy: 0.9944
Epoch 10/10
313/313 [==============================] - 

NameError: name 'classfication_report' is not defined

In [7]:
prediction = model.predict(test_padded)

print(prediction[:10])
for value in prediction:
    if value[0]>0.5:
        value[0]=1
    else:
        value[0]=0
print(accuracy_score(Y_test,prediction))
print(classification_report(Y_test,prediction))

313/313 [==============================] - 4s 13ms/step
[[3.3430302e-07]
 [9.9961942e-01]
 [9.3474811e-01]
 [1.3406038e-03]
 [1.4102313e-04]
 [9.7864622e-01]
 [9.9997783e-01]
 [1.6414369e-02]
 [9.9575210e-01]
 [9.9973077e-01]]
0.8338
              precision    recall  f1-score   support

           0       0.88      0.78      0.82      5036
           1       0.80      0.89      0.84      4964

    accuracy                           0.83     10000
   macro avg       0.84      0.83      0.83     10000
weighted avg       0.84      0.83      0.83     10000



In [61]:
messages = pd.read_csv("imdb_dataset.csv")

# messages = messages.iloc[:300,:]
corpus_words = []
corpus_sentences = []
WL =  WordNetLemmatizer()
for index in range(len(messages)):
  if index%1000==0:
     print(f"running index is {index}")
  html_tag_remover = re.sub("<.*>"," ",messages["review"][index])
  filtered_words = re.sub("\W"," ",html_tag_remover)
  message = filtered_words.lower()
  words_lst = [WL.lemmatize(word) for word in nltk.word_tokenize(message) if word not in stopwords.words('english')]
  document = " ".join(words_lst)
  corpus_sentences.append(document)
  corpus_words.append(words_lst)



running index is 0
running index is 1000
running index is 2000
running index is 3000
running index is 4000
running index is 5000
running index is 6000
running index is 7000
running index is 8000
running index is 9000
running index is 10000
running index is 11000
running index is 12000
running index is 13000
running index is 14000
running index is 15000
running index is 16000
running index is 17000
running index is 18000
running index is 19000
running index is 20000
running index is 21000
running index is 22000
running index is 23000
running index is 24000
running index is 25000
running index is 26000
running index is 27000
running index is 28000
running index is 29000
running index is 30000
running index is 31000
running index is 32000
running index is 33000
running index is 34000
running index is 35000
running index is 36000
running index is 37000
running index is 38000
running index is 39000
running index is 40000
running index is 41000
running index is 42000
running index is 43000
r

In [63]:
import gensim
from keras.initializers import Constant
embed_dim =300
w2v_model=Word2Vec(sentences=corpus_words,vector_size=embed_dim,window=10,min_count=1)
w2v_model.train(corpus_words,epochs=10,total_examples=len(corpus_words))
vocab = w2v_model.wv.key_to_index

print(len(vocab))

vocab=list(vocab.keys())
word_vec_dict={}
for word in vocab:
  word_vec_dict[word]=w2v_model.wv.get_vector(word)

print(f"number of key-value pairs in dict is {len(word_vec_dict)}")

max_num_tokens = -1
for sentence in corpus_words:
    if len(sentence)>max_num_tokens:
        max_num_tokens = len(sentence)


    
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus_sentences)
vocab_size = len(tokenizer.word_index) + 1
tokenized_sequences = tokenizer.texts_to_sequences(corpus_sentences)
padded_sentences= pad_sequences(tokenized_sequences, maxlen=max_num_tokens, padding='post')



print(f"first padded sentence is {padded_sentences[0]}")





embed_matrix=np.zeros(shape=(vocab_size,embed_dim))
for word,i in tokenizer.word_index.items():
  if word in word_vec_dict.keys():
      embed_vector=word_vec_dict[word]
      if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
        embed_matrix[i]=embed_vector


print(f"first element in embed_matrix is:{embed_matrix[10][10]}")
print("vocab_size is:",vocab_size)
print("embed matrix shape is :",embed_matrix.shape)


le = LabelEncoder()
order =  ["negative","positive"]
le.fit(order)
Y=le.transform(messages['sentiment'])

x_train,x_test,y_train,y_test=train_test_split(padded_sentences,Y,test_size=0.20,random_state=42)


model = Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embed_dim,input_length=max_num_tokens,embeddings_initializer=Constant(embed_matrix)))
model.add(Bidirectional(LSTM(units=128)))
model.add(Dense(64,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

print(model.summary())
model.fit(x_train,y_train,epochs=3,verbose=1)
pred = model.predict(x_test)

pred = (pred > 0.5).astype(int)

print(type(pred))
print(pred[0])

print(accuracy_score(y_test,pred))
# print(accuracy_score(y_test,prediction))
# print(f"accuracy_score:{accuracy}")
# print(f"classification_report:{classification_report}")
print(classification_report(y_test,pred))

73253
number of key-value pairs in dict is 73253
first padded sentence is [    3   958  1076    51   177  2546   155  2568   109   529   473     8
    41   216   966    20   593   101    33    20  2420   685    91   230
  4020  2055   179   685  1119   685   692  2546   724   115    21   155
    35    80  3158  1453  2221    41  1383   139  1327   809  2546    86
  9200   188   443  1199   481   481  6080  6562  2331  2639 16915  4779
   316   497    14   156    15  7392   572   523  4779   494  1094  4936
   593   365   605  1241  1094   359    51  2546   108   356  3873  3304
   482  1188    14   822  4104   355     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0  